In [13]:
import fitz  # PyMuPDF

pdf_path = "./data/KUROSE, James - Redes de Computadores e a Internet_ uma abordagem top-down-Pearson (2013).pdf"

def convert_pdf_to_text(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def split_text_into_documents(text, chunk_size=500):
    words = text.split()
    chunks = [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

pdf_text = convert_pdf_to_text(pdf_path)
pdf_chunks = split_text_into_documents(pdf_text)

docs = [Document(content=chunk) for chunk in pdf_chunks]

In [14]:
from haystack.telemetry import tutorial_running
tutorial_running(27)

In [15]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
document_store = InMemoryDocumentStore()

In [16]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

In [17]:
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])

Batches:   0%|          | 0/24 [00:00<?, ?it/s]

738

In [18]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

In [19]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
retriever = InMemoryEmbeddingRetriever(document_store)

In [20]:
from typing import List
from haystack import Pipeline, component
from transformers import pipeline

# Inicializando o modelo de question-answering
model_name = 'pierreguillou/bert-large-cased-squad-v1.1-portuguese'
qa_pipeline = pipeline("question-answering", model=model_name)

@component
class QAPipelineComponent:
    """
    A component for question answering using a Hugging Face pipeline
    """
    def __init__(self, qa_pipeline):
        self.qa_pipeline = qa_pipeline

    @component.output_types(answers=List[dict])
    def run(self, query: str, documents: List[Document]):
        context = ' '.join([doc.content for doc in documents])
        print(f"Context: {context}")
        result = self.qa_pipeline(question=query, context=context)
        print(f"Answer: {result['answer']}")
        return {"answers": [{"answer": result["answer"], "score": result["score"]}]}


In [21]:
# Criando o pipeline de texto
basic_rag_pipeline = Pipeline()
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component(name="qa_component", instance=QAPipelineComponent(qa_pipeline))

In [22]:
# Conectando os componentes entre si
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "qa_component.documents")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - qa_component: QAPipelineComponent
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> qa_component.documents (List[Document])

In [23]:
question = "Quais as camadas do modelo OSI?"
response = basic_rag_pipeline.run({
    "text_embedder": {"text": question},
    "qa_component": {"query": question}
})

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Context: capacidade de modificar a realização de um serviço sem afetar outros componentes do sistema é outra vantagem importante da divisão em camadas. Camadas de protocolo Mas chega de linhas aéreas! Vamos agora voltar nossa atenção a protocolos de rede. Para prover uma es- trutura para o projeto, projetistas de rede organizam protocolos — e o hardware e o software de rede que os executam — em camadas. Cada protocolo pertence a uma das camadas, assim como cada função na arquitetura de linha aérea da Figura 1.22 pertencia a uma camada. Mais uma vez estamos interessados nos serviços que uma camada oferece à camada acima dela — denominado modelo de serviço. Assim como em nosso exemplo da linha aérea, cada camada provê seu serviço (1) executando certas ações dentro dela e (2) utilizando os serviços da camada diretamente abaixo dela. Por exemplo, os serviços providos pela camada n podem incluir entrega confiável de mensagens de uma extremidade da rede à outra, que pode ser implementada uti

In [24]:
# Acessando e exibindo a primeira resposta e sua pontuação
first_answer = response["qa_component"]["answers"][0]
answer = first_answer["answer"]
score = first_answer["score"]
print(f"Answer: {answer}")
print(f"Score: {score}")

Answer: .
Score: 0.24663789570331573
